In [ ]:
import numpy as np
import random
from google.colab import drive
import json
import time
import pandas as pd
import os
import sys
import librosa
import soundfile as sf
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
speaker_list = ['KENN',
 'MCUR',
 'JTAN',
 'MPOL',
 'MPUR',
 'ADIZ',
 'NJAT',
 'PMAR',
 'JLEE',
 'SAMF',
 'ZHIY']

In [ ]:
test_speaker = ['VKOW']

In [ ]:
audio_array = []
phoneme_array = []

In [ ]:
!cp -r /content/gdrive/MyDrive/nus-smc-corpus_48/ /content/

^C


In [ ]:
for sp in speaker_list:
  for sig in os.listdir("/content/nus-smc-corpus_48/"+str(sp)+"/read/"):
    if '.wav' in sig:
      song = sig.replace('.wav','')
      df = pd.read_csv("/content/nus-smc-corpus_48/"+str(sp)+"/read/"+str(song)+".txt", delimiter=" ", header=None, names=['start', 'end', 'phoneme'])
      audio, sr = librosa.load("/content/nus-smc-corpus_48/"+str(sp)+"/read/"+str(sig), sr=None)
      audio = audio[:int(df['end'].iloc[-1]*sr)]

      for idx, row in df.iterrows():
        for i in range(int(row['start']*100),int(row['end']*100)):
          reqd_audio = audio[int(i/100*sr):int((i+1)/100*sr)]
          spectrogram = librosa.stft(reqd_audio, n_fft=256, hop_length=256, center=False, win_length=256)
          M = abs(spectrogram)
          audio_array.append(M)
          phoneme_array.append(row['phoneme'])

In [ ]:
# one_hot = np.eye(40)

In [ ]:
# one_hot_dict = {}
# for idx, ph in enumerate(set(phoneme_array)):
#   one_hot_dict[ph] = one_hot[idx,:]

In [ ]:
params = {'learning_rate':0.1, 'verbose':True}

In [ ]:
final_audio = np.array(audio_array)
final_audio = final_audio.reshape(final_audio.shape[0], final_audio.shape[1])

In [ ]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(phoneme_array)
label_encoded_y = label_encoder.transform(phoneme_array)

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(final_audio, label_encoded_y, test_size=0.25, random_state=11755)

In [ ]:
eval_set = [(train_X, train_Y), (test_X, test_Y)]
model = xgboost.XGBClassifier(**params)
model.fit(train_X, train_Y, early_stopping_rounds=2, eval_metric=["merror"], eval_set=eval_set, verbose=True)

[0]	validation_0-merror:0.584538	validation_1-merror:0.585931
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 2 rounds.
[1]	validation_0-merror:0.570385	validation_1-merror:0.573434
[2]	validation_0-merror:0.563907	validation_1-merror:0.566228
[3]	validation_0-merror:0.560344	validation_1-merror:0.562518
[4]	validation_0-merror:0.557531	validation_1-merror:0.560495
[5]	validation_0-merror:0.555647	validation_1-merror:0.558124
[6]	validation_0-merror:0.554209	validation_1-merror:0.556838
[7]	validation_0-merror:0.552754	validation_1-merror:0.555673
[8]	validation_0-merror:0.551057	validation_1-merror:0.553985
[9]	validation_0-merror:0.549981	validation_1-merror:0.552633
[10]	validation_0-merror:0.548865	validation_1-merror:0.551628
[11]	validation_0-merror:0.547793	validation_1-merror:0.550744
[12]	validation_0-merror:0.546481	validation_1-merror:0.549619
[13]	validation_0-merror:0.54

XGBClassifier(objective='multi:softprob', verbose=True)

In [ ]:
# fit model no training data
model = xgboost.XGBClassifier(**params)
model.fit(final_audio, label_encoded_y, verbose=True)

In [ ]:
model.save_model("/content/gdrive/MyDrive/xgboost_model.json")

In [ ]:
# new_phoneme = []
# for ph in phoneme_array:
#   new_phoneme.append(int(np.where(ph==1.)[0]))

In [ ]:
preds = model.predict(final_audio[:1000])

In [ ]:
sum(label_encoded_y[:1000] == preds)

525

In [ ]:
525/1000

0.525

In [ ]:
audio_array_test = []
phoneme_array_test = []

In [ ]:
for sp in test_speaker:
  for sig in os.listdir("/content/nus-smc-corpus_48/"+str(sp)+"/read/"):
    if '.wav' in sig:
      song = sig.replace('.wav','')
      df = pd.read_csv("/content/nus-smc-corpus_48/"+str(sp)+"/read/"+str(song)+".txt", delimiter=" ", header=None, names=['start', 'end', 'phoneme'])
      audio, sr = librosa.load("/content/nus-smc-corpus_48/"+str(sp)+"/read/"+str(sig), sr=None)
      audio = audio[:int(df['end'].iloc[-1]*sr)]

      for idx, row in df.iterrows():
        for i in range(int(row['start']*100),int(row['end']*100)):
          reqd_audio = audio[int(i/100*sr):int((i+1)/100*sr)]
          spectrogram = librosa.stft(reqd_audio, n_fft=256, hop_length=256, center=False, win_length=256)
          M = abs(spectrogram)
          audio_array_test.append(M)
          phoneme_array_test.append(row['phoneme'])

In [ ]:
final_audio_test = np.array(audio_array_test)
final_audio_test = final_audio_test.reshape(final_audio_test.shape[0], final_audio_test.shape[1])
label_encoded_y_test = label_encoder.transform(phoneme_array_test)

In [ ]:
preds_test = model.predict(final_audio_test)

In [ ]:
sum(preds_test==label_encoded_y_test)/len(label_encoded_y_test)

0.47689965089504566

In [ ]:
preds

array([30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30,
       30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 28, 28, 28, 28, 28, 28, 28,
       28, 28, 28, 28, 28, 28, 28, 22,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  2,  2,  2, 20, 20,  2, 20,  2, 22,  2,  2, 10, 10,  2,  2,
        2, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 22, 22, 22, 22, 22, 22,
       22, 22,  2,  2,  2,  2,  5,  5,  5,  5,  5, 10, 12, 17, 17, 17, 22,
        5, 30, 30, 30, 30